In [1]:
param_date = '20240706'

In [2]:
import os

list_file = sorted(list(os.walk(f'./{param_date}/4.model_2/'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['make_1.AC.xlsx',
 'make_10.American LaFrance.xlsx',
 'make_100.Lamborghini.xlsx',
 'make_101.Lancia.xlsx',
 'make_102.Land Rover.xlsx',
 'make_103.Lexus.xlsx',
 'make_104.Lincoln.xlsx',
 'make_105.Lion.xlsx',
 'make_106.Lotus.xlsx',
 'make_107.MG.xlsx',
 'make_108.Mack.xlsx',
 'make_109.Maple Leaf Truck.xlsx',
 'make_11.American Motors.xlsx',
 'make_110.Maserati.xlsx',
 'make_111.Maybach.xlsx',
 'make_112.Mazda.xlsx',
 'make_113.Mercedes-Benz.xlsx',
 'make_114.Mercury.xlsx',
 'make_115.Merkur.xlsx',
 'make_116.Metropolitan.xlsx',
 'make_117.Mini.xlsx',
 'make_118.Mitsubishi.xlsx',
 'make_119.Mitsubishi Fuso.xlsx',
 'make_12.Aston Martin.xlsx',
 'make_120.Mobility Ventures.xlsx',
 'make_121.Monaco Coach.xlsx',
 'make_122.Motor Coach Industries.xlsx',
 'make_123.Nash.xlsx',
 'make_124.National RV.xlsx',
 'make_125.Neoplan.xlsx',
 'make_126.New Flyer.xlsx',
 'make_127.Newmar.xlsx',
 'make_128.Nexus.xlsx',
 'make_129.Nissan.xlsx',
 'make_13.Asuna.xlsx',
 'make_130.Nova Bus Corporation.xl

In [3]:
file = list_file[0]

file

'make_1.AC.xlsx'

In [4]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [5]:
input_ = pd.read_excel(f'./{param_date}/4.model_2/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['Year'] = input_['Year'].astype(int)
input_['Make No'] = input_['Make No'].astype(int)
input_['Model No'] = input_['Model No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Model'].tolist()

总数量：3


['Shelby Cobra', 'Shelby Cobra', 'Shelby Cobra']

In [6]:
a = 0

a

0

In [7]:
crawler_status = 'error'

crawler_status

'error'

In [8]:
import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_ua, get_proxy

data = {'cssClass': 'form-control',
        'dropdownType': '3',
        'selectedValue': '',
        'vehicleAttributes': {'Make': f'''{input_.loc[a, 'Make']}''',
                              'MakeID': f'''{input_.loc[a, 'Make Code']}''',
                              'ModelID': f'''{input_.loc[a, 'Model Code']}''',
                              'SkinID': '1',
                              'Year': f'''{input_.loc[a, 'Year']}'''}}

resp = requests.post('https://www.dormanproducts.com/ApplicationGuideService.asmx/BuildDropdown',
                     data=json.dumps(data),
                     headers=get_ua(**{'Content-Type': 'application/json; charset=UTF-8'}),
                     proxies=get_proxy(),
                     timeout=(10, 10)).text

resp

'{"d":"\\u003cselect id=\\"parttypeDropdown\\" name=\\"parttypeDropdown\\" onchange=\\"showResults(this);\\" class=\\"form-control\\"\\u003e\\n\\u003coption value=\\"0\\"  disabled\\u003e{0}\\u003c/option\\u003e\\n\\u003coption value=\\"10420\\"  \\u003eBrake Adjusting Screw Spring Kit\\u003c/option\\u003e\\n\\u003coption value=\\"1572\\"  \\u003eTurn Signal Cam\\u003c/option\\u003e\\n\\u003c/select\\u003e"}'

In [9]:
json_ = json.loads(resp.strip())

json_

{'d': '<select id="parttypeDropdown" name="parttypeDropdown" onchange="showResults(this);" class="form-control">\n<option value="0"  disabled>{0}</option>\n<option value="10420"  >Brake Adjusting Screw Spring Kit</option>\n<option value="1572"  >Turn Signal Cam</option>\n</select>'}

In [10]:
from bs4 import BeautifulSoup
from lxml import etree

soup = BeautifulSoup(json_['d'].strip(), 'lxml')
html = etree.HTML(str(soup))

with open('./type_1.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <select class="form-control" id="parttypeDropdown" name="parttypeDropdown" onchange="showResults(this);">
   <option disabled="" value="0">
    {0}
   </option>
   <option value="10420">
    Brake Adjusting Screw Spring Kit
   </option>
   <option value="1572">
    Turn Signal Cam
   </option>
  </select>
 </body>
</html>



In [11]:
list_type_code = html.xpath('//select[@id="parttypeDropdown"]/option/@value')[1:]

list_type_code

['10420', '1572']

In [12]:
len(list_type_code) == 0

False

In [13]:
list_type = html.xpath('//select[@id="parttypeDropdown"]/option/text()')[1:]

list_type

['Brake Adjusting Screw Spring Kit', 'Turn Signal Cam']

In [14]:
df_temp = pd.DataFrame({'Year': input_.loc[a, 'Year'],
                        'Make No': input_.loc[a, 'Make No'],
                        'Make': input_.loc[a, 'Make'],
                        'Make Code': input_.loc[a, 'Make Code'],
                        'Model No': input_.loc[a, 'Model No'],
                        'Model': input_.loc[a, 'Model'],
                        'Model Code': input_.loc[a, 'Model Code'],
                        'Type No': [i+1 for i in range(len(list_type))],
                        'Type': list_type,
                        'Type Code': list_type_code,
                        'Type Url': [f'https://www.dormanproducts.com/gsearch.aspx?type=keyword&q={type_}' for type_ in list_type]})

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,Year,Make No,Make,Make Code,Model No,Model,Model Code,Type No,Type,Type Code,Type Url
0,1971,1,AC,457,1,Shelby Cobra,21378,1,Brake Adjusting Screw Spring Kit,10420,https://www.dormanproducts.com/gsearch.aspx?ty...
1,1971,1,AC,457,1,Shelby Cobra,21378,2,Turn Signal Cam,1572,https://www.dormanproducts.com/gsearch.aspx?ty...


In [15]:
crawler_status = 'ok'

crawler_status

'ok'

In [16]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Year': input_.loc[a, 'Year'],
                             'Make No': input_.loc[a, 'Make No'],
                             'Make': input_.loc[a, 'Make'],
                             'Make Code': input_.loc[a, 'Make Code'],
                             'Model No': input_.loc[a, 'Model No'],
                             'Model': input_.loc[a, 'Model'],
                             'Model Code': input_.loc[a, 'Model Code']}])

    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [17]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Year', 'Make No', 'Model No', 'Type No'],
                                                ascending=[False, True, True, True],
                                                ignore_index=True)
    output_correct.to_excel('./test_type_1.xlsx', index=False)

output_correct

,Year,Make No,Make,Make Code,Model No,Model,Model Code,Type No,Type,Type Code,Type Url
0,1971,1,AC,457,1,Shelby Cobra,21378,1,Brake Adjusting Screw Spring Kit,10420,https://www.dormanproducts.com/gsearch.aspx?ty...
1,1971,1,AC,457,1,Shelby Cobra,21378,2,Turn Signal Cam,1572,https://www.dormanproducts.com/gsearch.aspx?ty...


In [18]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['Year', 'Make No', 'Model No'],
                                            ascending=[False, True, True],
                                            ignore_index=True)
    output_error.to_excel('./test_type_1_error.xlsx', index=False)

output_error

""
